<a href="https://colab.research.google.com/github/Jagrutiti/openfoodfacts-ai/blob/circular-net-model/circular_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import pathlib
import cv2
import logging
logging.disable(logging.WARNING)

import matplotlib
import matplotlib.pyplot as plt

import numpy as np
from six import BytesIO
from PIL import Image
from six.moves.urllib.request import urlopen

import tensorflow as tf
import tensorflow_hub as hub

from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

tf.gfile = tf.io.gfile

%matplotlib inline

tf.get_logger().setLevel('ERROR')

ALL_MODELS = {
'material_model' : '/content/pre-trained-models/material_model/saved_model/saved_model/',
'material_form_model' : '/content/pre-trained-models/material_form/saved_model/saved_model',
'plastic_model' : '/content/pre-trained-models/plastic_type/saved_model/saved_model/'
}

# path to an image
IMAGES_FOR_TEST = {
  'Image1' : 'models/official/projects/waste_identification_ml/pre_processing/config/sample_images/image_2.png'
}

def normalize_image(image,
                    offset=(0.485, 0.456, 0.406),
                    scale=(0.229, 0.224, 0.225)):
  """Normalizes the image to zero mean and unit variance."""
  with tf.name_scope('normalize_image'):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    offset = tf.constant(offset)
    offset = tf.expand_dims(offset, axis=0)
    offset = tf.expand_dims(offset, axis=0)
    image -= offset

    scale = tf.constant(scale)
    scale = tf.expand_dims(scale, axis=0)
    scale = tf.expand_dims(scale, axis=0)
    image /= scale
    return image

    
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: the file path to the image

  Returns:
    uint8 numpy array with shape (1, h, w, 3)
  """
  image = None
  if(path.startswith('http')):
    response = urlopen(path)
    image_data = response.read()
    image_data = BytesIO(image_data)
    image = Image.open(image_data)
  else:
    image_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(image_data))

  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (1, im_height, im_width, 3)).astype(np.uint8)


def build_inputs_for_segmentation(image):
  """Builds segmentation model inputs for serving."""
  # Normalizes image with mean and std pixel values.
  image = normalize_image(image)
  return image

  # @title Model Selection { display-mode: "form", run: "auto" }
model_display_name = 'material_form_model' # @param ['material_model','material_form_model','plastic_model']
model_handle = ALL_MODELS[model_display_name]

print('Selected model:'+ model_display_name)
print('Model Handle at TensorFlow Hub: {}'.format(model_handle))

# @title Labels for the above model { display-mode: "form", run: "auto" }

if model_display_name == 'material_model':
  PATH_TO_LABELS = '/content/models/config/data/material_labels.pbtxt'
elif model_display_name == 'material_form_model':
  PATH_TO_LABELS = '/content/models/config/data/material_form_labels.pbtxt'
elif model_display_name == 'plastic_model':
  PATH_TO_LABELS = '/content/models/config/data/plastic_type_labels.pbtxt'

print('Labels selected for',model_display_name)
print('\n')
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)
category_index


print('loading model...')
hub_model = hub.load(model_handle)
print('model loaded!')

#@title Image Selection (don't forget to execute the cell!) { display-mode: "form"}
selected_image = 'Image1' # @param ['Image1']
flip_image_horizontally = True #@param {type:"boolean"}
convert_image_to_grayscale = False #@param {type:"boolean"}

image_path = IMAGES_FOR_TEST[selected_image]
image_np = load_image_into_numpy_array(image_path)

# Flip horizontally
if(flip_image_horizontally):
  image_np[0] = np.fliplr(image_np[0]).copy()

# Convert image to grayscale
if(convert_image_to_grayscale):
  image_np[0] = np.tile(
    np.mean(image_np[0], 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

print('min:',np.min(image_np[0]), 'max:', np.max(image_np[0]))
plt.figure(figsize=(24,32))
plt.imshow(image_np[0])
plt.show()

# get an input size of images on which an Instance Segmentation model is trained
hub_model_fn = hub_model.signatures["serving_default"]
height=hub_model_fn.structured_input_signature[1]['inputs'].shape[1]
width = hub_model_fn.structured_input_signature[1]['inputs'].shape[2]
input_size = (height, width)
print(input_size)

# apply pre-processing functions which were applied during training the model
image_np_cp = cv2.resize(image_np[0], input_size[::-1], interpolation = cv2.INTER_AREA)
image_np = build_inputs_for_segmentation(image_np_cp)
image_np = tf.expand_dims(image_np, axis=0)
image_np.get_shape()

# display pre-processed image
plt.figure(figsize=(24,32))
plt.imshow(image_np[0])
plt.show()

# running inference
results = hub_model_fn(image_np)

# different object detection models have additional results
# all of them are explained in the documentation
result = {key:value.numpy() for key,value in results.items()}
print(result.keys())

# selecting parameters for visualization
label_id_offset = 0
min_score_thresh =0.6
use_normalized_coordinates=True

if use_normalized_coordinates:
  # Normalizing detection boxes
  result['detection_boxes'][0][:,[0,2]] /= height
  result['detection_boxes'][0][:,[1,3]] /= width

# Visualize detection and masks
if 'detection_masks' in result:
  # we need to convert np.arrays to tensors
  detection_masks = tf.convert_to_tensor(result['detection_masks'][0])
  detection_boxes = tf.convert_to_tensor(result['detection_boxes'][0])

  detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes,
              image_np.shape[1], image_np.shape[2])
  detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                      np.uint8)

  result['detection_masks_reframed'] = detection_masks_reframed.numpy()
viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_cp,
      result['detection_boxes'][0],
      (result['detection_classes'][0] + label_id_offset).astype(int),
      result['detection_scores'][0],
      category_index=category_index,
      use_normalized_coordinates=use_normalized_coordinates,
      max_boxes_to_draw=200,
      min_score_thresh=min_score_thresh,
      agnostic_mode=False,
      instance_masks=result.get('detection_masks_reframed', None),
      line_thickness=2)

plt.figure(figsize=(24,32))
plt.imshow(image_np_cp)
plt.show()

# collecting all masks and saving

mask_count = np.sum(result['detection_scores'][0] >= min_score_thresh)
print('Total number of objects found are:', mask_count)
mask = np.zeros_like(detection_masks_reframed[0])
for i in range(mask_count):
  if result['detection_scores'][0][i] >= min_score_thresh:
    mask += detection_masks_reframed[i]

mask = tf.clip_by_value(mask, 0,1)
plt.figure(figsize=(24,32))
plt.imshow(mask,cmap='gray')
plt.show()




Selected model:material_form_model
Model Handle at TensorFlow Hub: /content/pre-trained-models/material_form/saved_model/saved_model
Labels selected for material_form_model


loading model...


OSError: ignored

In [ ]:
! python --version

! python3 -c 'import tensorflow as tf; print(tf.__version__)'  # for Python 3

Python 3.7.15
2.9.2
